<a href="https://colab.research.google.com/github/pandeyanuradha/Structured-sentiment-analysis-using-Dependency-Graphs/blob/main/BERT%20as%20a%20service%20(BAAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Links 
- https://medium.com/@aayanthi/bert-as-service-with-google-colab-e235472e108d
- https://github.com/Sougotadayglo/Bert-serving-client
- https://towardsdatascience.com/nlp-extract-contextualized-word-embeddings-from-bert-keras-tf-67ef29f60a7b


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import json
import ast
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.layers import Input, Dense
from keras.models import Model

## TF1


In [ ]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
!pip install bert-serving-server

     |████████████████████████████████| 61 kB 244 kB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=72ba69f49a4fc21f30164b44117efea9f48ed8152b0a3a4b41a395d041be2d5a
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip; unzip uncased_L-12_H-768_A-12.zip

--2021-11-19 06:45:29--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 66.102.1.128, 142.251.5.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  86.4MB/s    in 4.5s    

2021-11-19 06:45:33 (86.4 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
!nohup bert-serving-start -max_seq_len=128 -model_dir=uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
!pip install bert-serving-client

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
sentences_list = ["Make sure to use Tensorflow 1", "Enable GPU support", 
                  "This is a very easy way of getting BERT embeddings.",
                  "Natural Language Processing is interesting", 
                  "Have a good day.", 
                  "Bye Bye"]

embeddings = bc.encode(sentences_list)
embeddings

## TF2


In [ ]:
%tensorflow_version 2.x

Create a virtual environment with TF1.x and install the BERT server there.
BERT-client can be intalled in TF2 and requests can be made to the server.

In [ ]:
!pip3 install virtualenv
!virtualenv myenv
!source /content/myenv/bin/activate myenv; pip install tensorflow==1.14; pip install bert-serving-server; wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip; unzip uncased_L-12_H-768_A-12.zip; nohup bert-serving-start -max_seq_len=128 -model_dir=uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
!pip install bert-serving-client

In [ ]:
from bert_serving.client import BertClient

In [ ]:
bc = BertClient()

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Formal projects/Dependency Parsing/BERT/enBERT.csv")

In [ ]:
data.head()

In [ ]:
n = data.shape[0] 
data['embeddings'] = 0.0
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
for i in range(n):
  data['Source'][i] = ast.literal_eval(data['Source'][i])[0]
  data['Target'][i] = ast.literal_eval(data['Target'][i])[0]
  data['Polar_expression'][i] = ast.literal_eval(data['Polar_expression'][i])[0]

In [ ]:
data.head()

In [ ]:
ss1 = data['Source'].tolist()
ss2 = data['Target'].tolist()
ss3 = data['Polar_expression'].tolist()

In [ ]:
src = pd.DataFrame({'src' : ss1})
tar = pd.DataFrame({'tar' : ss2})
pe = pd.DataFrame({'pe' : ss3})

src['embeddings'] = 0.0
tar['embeddings'] = 0.0
pe['embeddings'] = 0.0

In [ ]:
src

In [ ]:
src = src.astype('object')
tar = tar.astype('object')
pe = pe.astype('object')

In [ ]:
# Source

sentences_list = src['src'].to_list()

embeddings = bc.encode(sentences_list)
embeddings

In [ ]:
for i in range(len(embeddings)):
  src['embeddings'][i] = list(embeddings[i])

In [ ]:
# Target

sentences_list = tar['tar'].to_list()

embeddings = bc.encode(sentences_list)
embeddings

In [ ]:
for i in range(len(embeddings)):
  tar['embeddings'][i] = list(embeddings[i])

In [ ]:
# PE

sentences_list = pe['pe'].to_list()

embeddings = bc.encode(sentences_list)
embeddings

In [ ]:
for i in range(len(embeddings)):
  pe['embeddings'][i] = list(embeddings[i])

In [ ]:
src

In [ ]:
tar

In [ ]:
pe

## MLP

In [ ]:
len(src['embeddings'][0])

In [ ]:
len(src['embeddings'][0] + tar['embeddings'][0] + pe['embeddings'][0])

In [ ]:
data.head()

In [ ]:
data = data.astype('object')

In [ ]:
for i in range(data.shape[0]):
  data['embeddings'][i] = (src['embeddings'][i] + tar['embeddings'][i] + pe['embeddings'][i])

In [ ]:
data.head()

In [ ]:
X = data['embeddings']
y = data['Polarity']

In [ ]:
label_encoder = preprocessing.LabelEncoder()
data['Polarity']= label_encoder.fit_transform(data['Polarity'])

In [ ]:
X = np.vstack(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 40)

In [ ]:
# X_train = pd.DataFrame(X_train)
# X_test = pd.DataFrame(X_test)

In [ ]:
num_classes = 1

In [ ]:
y.shape

In [ ]:
inp = Input(shape=(2304,)) # you have 2304 features
hidden = Dense(100, activation = 'sigmoid')(inp)  # 10 neurons in your hidden layer
out = Dense(num_classes, activation = 'softmax')(hidden)  

In [ ]:
model = Model(inputs=[inp], outputs=[out])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# https://stackoverflow.com/questions/21046417/python-conversion-of-list-of-arrays-to-2d-array/32787788

In [ ]:
y.unique()

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs = 10, batch_size = 512)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()